In [18]:
import os 
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [19]:
import json

question_info = []

with open("/home2/xjw/FastChat/fastchat/llm_judge/data/mt_bench/question.jsonl", "r") as f:
    for line in f:
        question_info.append(json.loads(line))

In [20]:
len(question_info)

80

In [21]:
question_info[1]

{'question_id': 82,
 'category': 'writing',
 'turns': ["Draft a professional email seeking your supervisor's feedback on the 'Quarterly Financial Report' you prepared. Ask specifically about the data analysis, presentation style, and the clarity of conclusions drawn. Keep the email short and to the point.",
  'Take a moment to evaluate and critique your own response.']}

In [22]:
type(question_info[1]['question_id'])

int

In [23]:
question_info_reverse = {}

for i in range(len(question_info)):
  s = ""
  question_info_reverse[question_info[i]['question_id']] = question_info[i]['turns']    

In [7]:
question_info_reverse[81]

['Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.',
 'Rewrite your previous response. Start every sentence with the letter A.']

In [8]:
prompt_template_info = {"name": "pair-v2-multi-turn", "type": "pairwise", "system_prompt": "Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user questions. You should choose the assistant that follows the user's instructions and answers the user's questions better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. You should focus on who provides a better answer to the second user question. Begin your evaluation by comparing the responses of the two assistants and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: \"[[A]]\" if assistant A is better, \"[[B]]\" if assistant B is better, and \"[[C]]\" for a tie.", "prompt_template": "<|The Start of Assistant A's Conversation with User|>\n\n### User:\n{question_1}\n\n### Assistant A:\n{answer_a_1}\n\n### User:\n{question_2}\n\n### Assistant A:\n{answer_a_2}\n\n<|The End of Assistant A's Conversation with User|>\n\n\n<|The Start of Assistant B's Conversation with User|>\n\n### User:\n{question_1}\n\n### Assistant B:\n{answer_b_1}\n\n### User:\n{question_2}\n\n### Assistant B:\n{answer_b_2}\n\n<|The End of Assistant B's Conversation with User|>", "description": "Prompt for multi-turn general questions", "category": "general", "output_format": "[[A]]"}

In [9]:
prompt_template_info.keys()

dict_keys(['name', 'type', 'system_prompt', 'prompt_template', 'description', 'category', 'output_format'])

In [10]:
prompt_template_info["system_prompt"]

'Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user questions. You should choose the assistant that follows the user\'s instructions and answers the user\'s questions better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. You should focus on who provides a better answer to the second user question. Begin your evaluation by comparing the responses of the two assistants and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]" if assistant B is better,

In [11]:
prompt_template_info["prompt_template"]

"<|The Start of Assistant A's Conversation with User|>\n\n### User:\n{question_1}\n\n### Assistant A:\n{answer_a_1}\n\n### User:\n{question_2}\n\n### Assistant A:\n{answer_a_2}\n\n<|The End of Assistant A's Conversation with User|>\n\n\n<|The Start of Assistant B's Conversation with User|>\n\n### User:\n{question_1}\n\n### Assistant B:\n{answer_b_1}\n\n### User:\n{question_2}\n\n### Assistant B:\n{answer_b_2}\n\n<|The End of Assistant B's Conversation with User|>"

In [12]:
prompt_template_info["output_format"]

'[[A]]'

In [13]:
prompt_template = prompt_template_info["system_prompt"] + "\n\n" + prompt_template_info["prompt_template"]

print(prompt_template)

Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user questions. You should choose the assistant that follows the user's instructions and answers the user's questions better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. You should focus on who provides a better answer to the second user question. Begin your evaluation by comparing the responses of the two assistants and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]" if assistant B is better, an

In [5]:
from datasets import load_dataset

mt_bench = load_dataset("/home2/xjw/mt_bench_human_judgements")

print(len(mt_bench['human']))

/home2/xjw/anaconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3355


In [15]:
mt_bench['human'][0]['conversation_a']

mt_bench['human'].features

{'question_id': Value(dtype='int64', id=None),
 'model_a': Value(dtype='string', id=None),
 'model_b': Value(dtype='string', id=None),
 'winner': Value(dtype='string', id=None),
 'judge': Value(dtype='string', id=None),
 'conversation_a': [{'content': Value(dtype='string', id=None),
   'role': Value(dtype='string', id=None)}],
 'conversation_b': [{'content': Value(dtype='string', id=None),
   'role': Value(dtype='string', id=None)}],
 'turn': Value(dtype='int64', id=None)}

In [16]:
instructions = []

for i in range(len(mt_bench['human'])):
    system_part = """Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user questions. You should choose the assistant that follows the user's instructions and answers the user's questions better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. You should focus on who provides a better answer to the second user question. Begin your evaluation by comparing the responses of the two assistants and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible."""
    
    question_content = question_info_reverse[mt_bench['human'][i]['question_id']]  
    
    part_a = "<|The Start of Assistant A's Conversation with User|>\n\n"
    for j in range(len(question_content)):
        part_a = part_a + "### User:\n" + question_content[j] + "\n\n### Assistant A:\n" + mt_bench['human'][i]['conversation_a'][j * 2 +1]['content'] + "\n\n"
    part_a = part_a + "<|The End of Assistant A's Conversation with User|>\n\n\n"
    
    part_b = "<|The Start of Assistant B's Conversation with User|>\n\n"
    for j in range(len(question_content)):
        part_b = part_b + "### User:\n" + question_content[j] + "\n\n### Assistant B:\n" + mt_bench['human'][i]['conversation_b'][j * 2 +1]['content'] + "\n\n"
    part_b = part_b + "<|The End of Assistant B's Conversation with User|>"
    
    instructions.append([system_part, (part_a + part_b)])
    
print(len(instructions))

3355


In [17]:
print(instructions[0][0])

Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user questions. You should choose the assistant that follows the user's instructions and answers the user's questions better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. You should focus on who provides a better answer to the second user question. Begin your evaluation by comparing the responses of the two assistants and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible.


In [7]:
from vllm import LLM, SamplingParams


model = "/home2/xjw/LLaMA-Factory/models/llama3_lora_sft"
llm = LLM(model=model, tensor_parallel_size=1)

2024-06-05 18:56:54,387	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 06-05 18:56:54 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='/home2/xjw/LLaMA-Factory/models/llama3_lora_sft', speculative_config=None, tokenizer='/home2/xjw/LLaMA-Factory/models/llama3_lora_sft', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/home2/xjw/LLaMA-Factory/models/llama3_lora_sft)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-05 18:57:02 model_runner.py:146] Loading model weights took 14.9595 GB
INFO 06-05 18:57:03 gpu_executor.py:83] # GPU blocks: 27895, # CPU blocks: 2048
INFO 06-05 18:57:06 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-05 18:57:06 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-05 18:57:11 model_runner.py:924] Graph capturing finished in 6 secs.


In [23]:
input_tmeplate =  """<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{input}\n\nDirectly output your final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]" if assistant B is better, and "[[C]]" for a tie.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"""

inputs = [ input_tmeplate.format(system=instruction[0], input=instruction[1]) for instruction in instructions]

In [24]:
print(inputs[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user questions. You should choose the assistant that follows the user's instructions and answers the user's questions better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. You should focus on who provides a better answer to the second user question. Begin your evaluation by comparing the responses of the two assistants and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible.<|eot_id|><|start_header_id|>user<|end_header_id|>

<|The Start of Assistant A's Conversation with User|>

In [25]:
stop_tokens = ["<|eot_id|>", "<|end_of_text|>"]
sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=2048, stop=stop_tokens)

completions = llm.generate(inputs, sampling_params=sampling_params)

generated_responses = [output.outputs[0].text for output in completions]

with open("mt_bench_output.json", "w") as f:
  json.dump(generated_responses, f)

Processed prompts:  68%|██████▊   | 2293/3355 [14:44<21:20,  1.21s/it, Generation Speed: 260.53 toks/s]

WARNING 06-05 17:57:39 scheduler.py:1077] Sequence group 2480 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts: 100%|██████████| 3355/3355 [28:44<00:00,  1.95it/s, Generation Speed: 488.76 toks/s]  


In [13]:
import json

with open("/home2/xjw/mt_bench_output.json", "r") as f:
  generated_responses = json.load(f)

In [ ]:
generated_responses

In [15]:
ans = []

for i in range(len(generated_responses)):
    if generated_responses[i].rfind("[[A]]") != -1:
        ans.append("model_a")
        continue
    elif generated_responses[i].rfind("[[B]]") != -1:
        ans.append("model_b")
        continue
    elif generated_responses[i].rfind("[[C]]") != -1:
        ans.append("tie")
        continue
    else:
        ans.append("No answer!")

In [ ]:
ans

In [ ]:
correct = 0

sum_a = 0
sum_c = 0
sum_b = 0
correct_a = 0
correct_b = 0
correct_c = 0
pred_a = 0
pred_b = 0
pred_c = 0


for i in range(len(ans)):
    pred = ans[i]
    ground_truth = mt_bench['human'][i]['winner']
    if pred == ground_truth:
        correct += 1
        if pred == "model_a":
            correct_a += 1
        elif pred == "model_b":
            correct_b += 1
        else:
            correct_c += 1
    
    if pred == "model_a":
        pred_a += 1
    elif pred == "model_b":
        pred_b += 1
    elif pred == "tie":
        pred_c += 1
    else:
        pass
        #print("No answer!")
    
    if ground_truth == "model_a":
        sum_a += 1
    elif ground_truth == "model_b":  
        sum_b += 1
    else:
        sum_c += 1

def calc_p(correct, pred):
    return correct/pred
def calc_r(correct, sum):
    return correct/sum
def calc_f1(p, r):
    return 2*p*r/(p+r)

p_a = calc_p(correct_a, pred_a)
r_a = calc_r(correct_a, sum_a)
f1_a = calc_f1(p_a, r_a)

p_b = calc_p(correct_b, pred_b)
r_b = calc_r(correct_b, sum_b)
f1_b = calc_f1(p_b, r_b)

p_c = calc_p(correct_c, pred_c)
r_c = calc_r(correct_c, sum_c)

if p_c + r_c == 0:
    f1_c = 0
else:
    f1_c = calc_f1(p_c, r_c)

    
print(f"Accuracy: {correct*100/len(ans):.2f}")
print(f"Precision: {(p_a + p_b + p_c)*100/3:.2f}")
print(f"Recall: {(r_a + r_b + r_c)*100/3:.2f}")
print(f"F1: {(f1_a + f1_b + f1_c)*100/3:.2f}")